In [23]:
!pip install pyyaml

In [24]:
import os
import fiftyone as fo
import yaml
from skimage import io, transform
import matplotlib.pyplot as plt 
import numpy as np

In [25]:
CURRENT_DIR = os.getcwd()
DATA_PATH = os.path.join(CURRENT_DIR, 'Dataset')
yaml_file = os.path.join(DATA_PATH, "data.yaml")
print(CURRENT_DIR) 

/home/jovyan/BA/detection


In [8]:
# Load the dataset from fiftyone
ds = fo.load_dataset("sellpy-test")

In [9]:
#limit = 10000
# get all ids
ds_temp = ds.match(fo.ViewField("Grounding_Dino") !=  None)
print(ds_temp)
ids = ds_temp.values("id")
# check weather the sample contains a logo
contains_logo = ["logo" in values for values in ds_temp.values("Grounding_Dino.detections.label")]
# get ids of sample containing a logo 
ids_contains_logo = np.array(ids)[contains_logo]
# filter the data set to contain only samples with a logo found
ds_with_logos = ds[ids_contains_logo]

Dataset:     sellpy-test
Media type:  image
Num samples: 204676
Sample fields:
    id:             fiftyone.core.fields.ObjectIdField
    filepath:       fiftyone.core.fields.StringField
    tags:           fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    lvl0:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    lvl1:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    lvl2:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    lvl3:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    size:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    color:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    fabric:         fiftyon

In [95]:
all_brands = ds_with_logos.count_values('brand.label')

In [78]:
total_fiftyone_images = sum(all_brands.values())

In [101]:
all_classes_in_fiftyone = list(all_brands.keys())
len(all_classes_in_fiftyone)

5177

In [102]:
with open(yaml_file, 'r') as file:
    yamlFile = yaml.safe_load(file)   

In [103]:
all_classes_in_yolo = (yamlFile['names'])
len(all_classes_in_yolo)

569

In [114]:
import re
def normalize_brands(brand_list):
    normalized = []
    for brand in brand_list:
        # Umwandlung in Kleinbuchstaben
        brand = brand.lower()
        # Entfernen aller Wörter nach dem ersten
        #clean_brand = re.sub(r'\s.*', '', brand)
        normalized.append(brand)
    return normalized

all_classes_in_fiftyone_normalized = normalize_brands(all_classes_in_fiftyone)
all_classes_in_yolo_normalized = normalize_brands(all_classes_in_yolo)
matches = set(all_classes_in_yolo_normalized).intersection(set(all_classes_in_fiftyone_normalized))

In [115]:
len(matches)

118

In [116]:
print(f'Total fiftyone Brands: {len(all_classes_in_fiftyone)}')
print(f'Total fiftyone images: {total_fiftyone_images}')

print(f'Total Logo3K Brands: {len(all_classes_in_yolo)}')

# Check if there are any matches
if matches:
    print("Total Matches found:", len(matches))
else:
    print("No matches found.")
model_fiftyone_labels_map = {all_classes_in_fiftyone[all_classes_in_fiftyone_normalized.index(b)]:all_classes_in_yolo_normalized.index(b) for b in matches}

print('...............................................')
idx = [all_classes_in_fiftyone_normalized.index(b) for b in matches]

all_brands[all_classes_in_fiftyone[3268]]
result_dict = {all_classes_in_fiftyone[key]:all_brands[all_classes_in_fiftyone[key]] for key in idx}
print(result_dict)
print('...............................................')
print(f'Total number of images for matches: {sum(result_dict.values())}')
print('...............................................')

Total fiftyone Brands: 5177
Total fiftyone images: 37507
Total Logo3K Brands: 569
Total Matches found: 118
...............................................
{'Old Navy': 7, 'Pepe Jeans': 25, 'RVCA': 1, 'Emilio': 5, 'Moncler': 3, 'Wonderbra': 3, 'Everlast': 4, 'Montane': 1, 'New Balance': 28, 'Ermenegildo Zegna': 1, 'Karhu': 25, 'Wrangler': 19, 'Missoni': 2, 'Rare': 2, 'Joules': 7, 'Imperial': 1, 'Frank Dandy': 7, 'Vince Camuto': 1, 'Hurley': 2, 'Calvin Klein Underwear': 1, "Arc'teryx": 1, 'Lonsdale': 3, 'Columbia': 18, 'Napapijri': 28, 'Marc by Marc Jacobs': 3, 'Happy Socks': 8, 'Corneliani': 1, 'Mexx': 33, 'The North Face': 168, 'Versace': 1, 'Tom Tailor': 63, 'Barbie': 3, 'Canada Goose': 2, 'Heat': 1, 'Schiesser': 3, 'Guess': 66, 'Etro': 3, 'Tommy Hilfiger': 366, 'Givova': 1, 'Uniqlo': 46, 'Ellesse': 87, 'Lacoste': 57, 'Izod': 3, 'Chaps': 1, 'Salomon': 21, 'Head': 5, 'Hugo Boss': 17, 'Akris': 1, 'Strellson': 2, 'Miss Sixty': 6, 'Seafolly': 2, 'Sergio Tacchini': 10, 'Abercrombie & Fitch